In [1]:
# !python -m pip freeze | grep scikit-learn
!python -m pip freeze | findstr scikit-learn
# 1.6.1 (1.5.0)

scikit-learn==1.5.0


In [3]:
!python -V  # 1.13.3 (1.10.13)

Python 3.10.18


In [4]:
import pickle
import pandas as pd

In [5]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [6]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [ ]:
year, month = 2023, 3
df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet')

In [ ]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [11]:
y_pred.std()

np.float64(6.247488852238703)

In [13]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [18]:
df_result = df[["ride_id"]]
df_result["prediction"] = y_pred
output_file = "prediction.parquet"

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

C:\Users\Manuel\AppData\Local\Temp\ipykernel_11392\1552131472.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_result["prediction"] = y_pred
